In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/MyDrive/ML/TrainingSignalGuitarTX.wav' '/content/input.wav'
!cp '/content/drive/MyDrive/ML/5153B_ReactIR/ToneX_5153B_ReactIR.wav' '/content/output.wav'

In [ ]:
!git clone https://github.com/KirillDeLutherSergeev/AmperiumNNM.git '/content/AmperiumNNM'

%cd 'AmperiumNNM/'

from AudioUtils import *
from Training import *
from ExportNNM import *
import matplotlib.pyplot as plt

In [ ]:
# EDIT THIS SECTION FOR USER INPUTS
#
name = 'EVH5153B_V1'
in_file = '/content/input.wav'
out_file = '/content/output.wav'

check_if_model_exists(name)

x_all, y_all = load_audio_data(in_file, out_file, 5, 0, 2)

x_train, y_train, x_test, y_test = prepare_dataset(x_all, y_all, numSteps = 2048, trainSize=0.8, overlap=256, batchSize=64)

In [ ]:
model = build_model(True, True, True, loss='mae', learningRate=0.008, epsilon=1e-08)

In [ ]:
train_model(x_train, y_train, x_test, y_test, model, 8)

export_model_to_nnm('models/'+name+'/'+name+'.nnm', 1, 0, model, 1)
export_model_to_nnm('/content/drive/MyDrive/ML/5153B_ReactIR/EVH5153B_V2.nnm', 1, 0, model, 1)